PyAims tutorial : programming with AIMS in Python language
==========================================================

This tutorial should work with Python 2 (2.6 or higher), and is also compatible with Python 3 (if pyaims is compiled in python3 mode).

AIMS is a C++ library, but has python language bindings: [**PyAIMS**](http://brainvisa.info/pyaims/sphinx/). This means that the C++ classes and functions can be used from python. 
This has many advantages compared to pure C++:

* Writing python scripts and programs is much easier and faster than C++: there is no fastidious and long compilation step.
* Scripts are more flexible, can be modified on-the-fly, etc
* It can be used interactively in a python interactive shell.
* As pyaims is actually C++ code called from python, it is still fast to execute complex algorithms. 
  There is obviously an overhead to call C++ from python, but once in the C++ layer, it is C++ execution speed.

A few examples of how to use and manipulate the main data structures will be shown here.

The data for the examples in this section can be downloaded here: [ftp://ftp.cea.fr/pub/dsv/anatomist/data/demo_data.zip](ftp://ftp.cea.fr/pub/dsv/anatomist/data/demo_data.zip). 
To use the examples directly, users should go to the directory where this archive was uncompressed, and then run ipython from this directory.
A cleaner alternative, especially if no write access is allowed on this data directory, is to make a symbolic link to the *data_for_anatomist* subdirectory

```bash
cd $HOME
mkdir bvcourse
cd bvcourse
ln -s <path_to_data>/data_for_anatomist .
ipython
```

Doing this in python:

To work smoothly with python2 or python3, let's use print():

In [1]:
from __future__ import print_function

In [2]:
import urllib2
import zipfile
import os
import os.path
import tempfile
# let's work in a temporary directory
tuto_dir = tempfile.mkdtemp(prefix='pyaims_tutorial_')
f = urllib2.urlopen('ftp://ftp.cea.fr/pub/dsv/anatomist/data/demo_data.zip')
demo_data = os.path.join(tuto_dir, 'demo_data.zip')
open(demo_data, 'w').write(f.read())
f.close()
older_cwd = os.getcwd()
os.chdir(tuto_dir)
f = zipfile.ZipFile(demo_data)
f.extractall()
del f
print('we are working in:', tuto_dir)

we are working in: /tmp/pyaims_tutorial_mMtDlu



Using data structures
---------------------

### Module importation

In python, the aimsdata library is available as the [soma.aims](index.html#module-soma.aims) module.


In [3]:
import soma.aims
# the module is actually soma.aims:
vol = soma.aims.Volume(100, 100, 100, dtype='int16')

or:

In [4]:
from soma import aims
# the module is available as aims (not soma.aims):
vol = aims.Volume(100, 100, 100, dtype='int16')
# in the following, we will be using this form because it is shorter.

### IO: reading and writing objects

Reading operations are accessed via a single [soma.aims.read()](pyaims_highlevel.html#soma.aims.read) function, and writing through a single [soma.aims.write()](pyaims_highlevel.html#soma.aims.write) function. 
[soma.aims.read()](pyaims_highlevel.html#soma.aims.read) function reads any object from a given file name, in any supported file format, and returns it:


In [5]:
from soma import aims
obj = aims.read('data_for_anatomist/subject01/subject01.nii')
print(obj.getSize())
obj2 = aims.read('data_for_anatomist/subject01/Audio-Video_T_map.nii')
print(obj2.getSize())
obj3 = aims.read('data_for_anatomist/subject01/subject01_Lhemi.mesh')
print(len(obj3.vertex(0)))
assert(obj.getSize() == [256, 256, 124, 1])
assert(obj2.getSize() == [53, 63, 46, 1])
assert(obj3.size() == 1 and len(obj3.vertex(0)) == 33837)

[ 256, 256, 124, 1 ]
[ 53, 63, 46, 1 ]
33837


The returned object can have various types according to what is found in the disk file(s).

Writing is just as easy. The file name extension generally determines the output format. 
An object read from a given format can be re-written in any other supported format, provided the format can actually store the object type.

In [6]:
from soma import aims
obj2 = aims.read('data_for_anatomist/subject01/Audio-Video_T_map.nii')
aims.write(obj2, 'Audio-Video_T_map.ima')
obj3 = aims.read('data_for_anatomist/subject01/subject01_Lhemi.mesh')
aims.write(obj3, 'subject01_Lhemi.gii')

<div class="topic">
<p><b>Exercise</b></p>
Write a little file format conversion tool
</div>

### Volumes

Volumes are array-like containers of voxels, plus a set of additional information kept in a header structure. 
In AIMS, the header structure is generic and extensible, and does not depend on a specific file format. 
Voxels may have various types, so a specific type of volume should be used for a specific type of voxel. 
The type of voxel has a code that is used to suffix the Volume type: [soma.aims.Volume_S16](pyaims_lowlevel.html#soma.aims.soma.aims.Volume_S16) for signed 16-bit ints, [soma.aims.Volume_U32](pyaims_lowlevel.html#soma.aims.soma.aims.Volume_U32) 
for unsigned 32-bit ints, [soma.aims.Volume_FLOAT](pyaims_lowlevel.html#soma.aims.Volume_FLOAT) for 32-bit floats, [soma.aims.Volume_DOUBLE](pyaims_lowlevel.html#soma.aims.soma.aims.Volume_DOUBLE) for 64-bit floats, [soma.aims.Volume_RGBA](pyaims_lowlevel.html#soma.aims.soma.aims.Volume_RGBA) for RGBA colors, etc.


#### Building a volume

In [7]:
 # create a 3D volume of signed 16-bit ints, of size 192x256x128
vol = aims.Volume(192, 256, 128, dtype='int16')
# fill it with zeros
vol.fill(0)
# set value 12 at voxel (100, 100, 60)
vol.setValue(12, 100, 100, 60)
# get value at the same position
x = vol.value(100, 100, 60)
print(x)
assert(x == 12)

12


In [8]:
# set the voxels size
vol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
print(vol.header())
assert(vol.header() == {'sizeX': 192, 'sizeY': 256, 'sizeZ': 128, 'sizeT': 1, 'voxel_size': [0.9, 0.9, 1.2, 1]})

{ 'sizeX' : 192, 'sizeY' : 256, 'sizeZ' : 128, 'sizeT' : 1, 'voxel_size' : [ 0.9, 0.9, 1.2, 1 ] }


![3D volume: value 12 at voxel (100, 100 ,60)](images/volume1.png "3D volume: value 12 at voxel (100, 100 ,60)")


#### Basic operations

Whole volume operations:

In [9]:
# multiplication, addition etc
vol *= 2
vol2 = vol * 3 + 12
print(vol2.value(100, 100, 60))
vol /= 2
vol3 = vol2 - vol - 12
print(vol3.value(100, 100, 60))
vol4 = vol2 * vol / 6
print(vol4.value(100, 100, 60))
assert(vol2.value(100, 100, 60) == 84)
assert(vol3.value(100, 100, 60) == 60)
assert(vol4.value(100, 100, 60) == 168)

84
60
168


Voxel-wise operations:

In [10]:
# fill the volume with the distance to voxel (100, 100, 60)
vs = vol.header()['voxel_size']
pos0 = (100 * vs[0], 100 * vs[1], 60 * vs[2]) # in millimeters
for z in range(vol.getSizeZ()):
    for y in range(vol.getSizeY()):
        for x in range(vol.getSizeX()):
            # get current position in an aims.Point3df structure, in mm
            p = aims.Point3df(x * vs[0], y * vs[1], z * vs[2])
            # get relative position to pos0, in voxels
            p -= pos0
            # distance: norm of vector p
            dist = p.norm()
            # set it into the volume
            vol.setValue(dist, x, y, z)
print(vol.value(100, 100, 60))
# save the volume
aims.write(vol, 'distance.nii')
assert(vol.value(100, 100, 60) == 0)

0


Now look at the *distance.nii* volume in Anatomist.

![Distance example](images/distance.png "Distance example")


<div class="topic">
<p><b>Exercise</b></p>

Make a program which loads the image *data_for_anatomist/subject01/Audio-Video_T_map.nii* and thresholds it so as to keep values above 3.
</div>


In [11]:
from soma import aims
vol = aims.read('data_for_anatomist/subject01/Audio-Video_T_map.nii')
print(vol.value(20, 20, 20) < 3. and vol.value(20, 20, 20) != 0.)
assert(vol.value(20, 20, 20) < 3. and vol.value(20, 20, 20) != 0.)
for z in range(vol.getSizeZ()):
    for y in range(vol.getSizeY()):
        for x in range(vol.getSizeX()):
            if vol.value(x, y, z) < 3.:
                vol.setValue(0, x, y, z)
print(vol.value(20, 20, 20))
aims.write(vol, 'Audio-Video_T_thresholded.nii')
assert(vol.value(20, 20, 20) == 0.)

True
0.0


![Thresholded Audio-Video T-map](images/threshold.png "Thresholded Audio-Video T-map")

<div class="topic">
<p><b>Exercise</b></p>

Make a program to dowsample the anatomical image *data_for_anatomist/subject01/subject01.nii* and keeps one voxel out of two in every direction.
</div>

In [12]:
from soma import aims
vol = aims.read('data_for_anatomist/subject01/subject01.nii')
# allocate a new volume with half dimensions
vol2 = aims.Volume(vol.getSizeX() / 2, vol.getSizeY() / 2, vol.getSizeZ() / 2, dtype='DOUBLE')
print(vol2.getSizeX())
assert(vol2.getSizeX() == 128)
# set the voxel size to twice it was in vol
vs = vol.header()['voxel_size']
vs2 = [x * 2 for x in vs]
vol2.header()['voxel_size'] = vs2
for z in range(vol2.getSizeZ()):
    for y in range(vol2.getSizeY()):
        for x in range(vol2.getSizeX()):
            vol2.setValue(vol.value(x*2, y*2, z*2), x, y, z)
print(vol.value(100, 100, 40))
print(vol2.value(50, 50, 20))
aims.write(vol2, 'resampled.nii')
assert(vol.value(100, 100, 40) == 775)
assert(vol2.value(50, 50, 20) == 775.)

128
775
775.0


![Downsampled anatomical image](images/resampled.png "Downsampled anatomical image")

The first thing that comes to mind when running these examples, is that they are *slow*. 
Indeed, python is an interpreted language and loops in any interpreted language are slow. 
In addition, accessing individually each voxel of the volume has the overhead of python/C++ bindings communications. 
The conclusion is that that kind of example is probably a bit too low-level, and should be done, when possible, by compiled libraries or specialized array-handling libraries. 
This is the role of **numpy**.

Accessing numpy arrays to AIMS volume voxels is supported:

In [13]:
import numpy
vol.fill(0)
arr = numpy.asarray(vol)
# set value 100 in a whole sub-volume
arr[60:120, 60:120, 40:80] = 100
# note that arr is a shared view to the volume contents,
# modifications will also affect the volume
print(vol.value(65, 65, 42))
print(vol.value(65, 65, 30))
aims.write(vol, "cube.nii")
assert(vol.value(65, 65, 42) == 100)
assert(vol.value(65, 65, 30) == 0)

100
0


![3D volume containing a cube](images/cube.png "3D volume containing a cube")

Now we can re-write the thresholding example using numpy:

In [14]:
from soma import aims
vol = aims.read('data_for_anatomist/subject01/Audio-Video_T_map.nii')
arr = numpy.asarray(vol)
arr[numpy.where(arr < 3.)] = 0.
print(vol.value(20, 20, 20))
aims.write(vol, 'Audio-Video_T_thresholded2.nii')
assert(vol.value(20, 20, 20) == 0)

0.0


Here, `arr < 3.` returns a boolean array with the same size as `arr`, and [numpy.where()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html) returns arrays of coordinates where the specified contition is true.

The distance example, using numpy, would like the following:


In [29]:
from soma import aims
import numpy
vol = aims.Volume(192, 256, 128, 'S16')
vol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
vs = vol.header()['voxel_size']
pos0 = (100 * vs[0], 100 * vs[1], 60 * vs[2]) # in millimeters
arr = numpy.asarray(vol)
# build arrays of coordinates for x, y, z
x, y, z = numpy.ogrid[0.:vol.getSizeX(), 0.:vol.getSizeY(), 0.:vol.getSizeZ()]
# get coords in millimeters
x *= vs[0]
y *= vs[1]
z *= vs[2]
# relative to pos0
x -= pos0[0]
y -= pos0[1]
z -= pos0[2]
# get norm, using numpy arrays broadcasting
arr[:, :, :, 0] = numpy.sqrt(x**2 + y**2 + z**2)

print(vol.value(100, 100, 60))
assert(vol.value(100, 100, 60) == 0)

# and save result
aims.write(vol, 'distance2.nii')

0


This example appears a bit more tricky, since we must build the coordinates arrays, but is way faster to execute, because all loops within the code are executed in compiled routines in numpy. 
One interesting thing to note is that this code is using the famous "array broadcasting" feature of numpy, where arrays of heterogeneous sizes can be combined, and the "missing" dimensions are extended.


#### Copying volumes or volumes structure, or building from an array

To make a deep-copy of a volume, use the copy constructor:


In [33]:
vol2 = aims.Volume(vol)
vol2.setValue(12, 100, 100, 60)
# now vol and vol2 have different values
print('vol.value(100, 100, 60):', vol.value(100, 100, 60))
assert(vol.value(100, 100, 60) == 0)
print('vol2.value(100, 100, 60):', vol2.value(100, 100, 60))
assert(vol2.value(100, 100, 60) == 12)

vol.value(100, 100, 60): 0
vol2.value(100, 100, 60): 12


If you need to build another, different volume, with the same structure and size, don't forget to copy the header part:

In [34]:
vol2 = aims.Volume(vol.getSizeX(), vol.getSizeY(), vol.getSizeZ(), vol.getSizeT(), 'FLOAT')
vol2.header().update(vol.header())
print(vol2.header())
assert(vol2.header() == {'sizeX': 192, 'sizeY': 256, 'sizeZ': 128, 'sizeT': 1, 'voxel_size': [0.9, 0.9, 1.2, 1]})

{ 'sizeX' : 192, 'sizeY' : 256, 'sizeZ' : 128, 'sizeT' : 1, 'voxel_size' : [ 0.9, 0.9, 1.2, 1 ] }


Important information can reside in the header, like voxel size, or coordinates systems and geometric transformations to other coordinates systems, 
so it is really very important to carry this information with duplicated or derived volumes.

You can also build a volume from a numpy array:

In [39]:
arr = numpy.array(numpy.diag(xrange(40)), dtype=numpy.float32).reshape(40, 40, 1) \
    + numpy.array(xrange(20), dtype=numpy.float32).reshape(1, 1, 20)
# WARNING: the array must be in Fortran ordering for AIMS, at leat at the moment
# whereas the numpy addition always returns a C-ordered array
arr = numpy.array(arr, order='F')
arr[10, 12, 3] = 25
vol = aims.Volume(arr)
print('vol.value(10, 12, 3):', vol.value(10, 12, 3))
assert(vol.value(10, 12, 3) == 25.)

# data are shared with arr
vol.setValue(35, 10, 15, 2)
print('arr[10, 15, 2]:', arr[10, 15, 2])
assert(arr[10, 15, 2] == 35.0)
arr[12, 15, 1] = 44
print('vol.value(12, 15, 1):', vol.value(12, 15, 1))
assert(vol.value(12, 15, 1) == 44.0)

vol.value(10, 12, 3): 25.0
arr[10, 15, 2]: 35.0
vol.value(12, 15, 1): 44.0


#### 4D volumes

4D volumes work just like 3D volumes. Actually all volumes are 4D in AIMS, but the last dimension is commonly of size 1. 
In [soma.aims.Volume_FLOAT.value](pyaims_lowlevel.html#soma.aims.Volume_FLOAT.value) and [soma.aims.Volume_FLOAT.setValue](pyaims_lowlevel.html#soma.aims.Volume_FLOAT.setValue) methods, only the first dimension is mandatory, 
others are optional and default to 0, but up to 4 coordinates may be used. In the same way, the constructor takes up to 4 dimension parameters:


In [42]:
from soma import aims
# create a 4D volume of signed 16-bit ints, of size 30x30x30x4
vol = aims.Volume(30, 30, 30, 4, 'S16')
# fill it with zeros
vol.fill(0)
# set value 12 at voxel (10, 10, 20, 2)
vol.setValue(12, 10, 10, 20, 2)
# get value at the same position
x = vol.value(10, 10, 20, 2)
print(x)
assert(x == 12)
# set the voxels size
vol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
print(vol.header())
assert(vol.header() == {'sizeX': 30, 'sizeY': 30, 'sizeZ': 30, 'sizeT': 4, 'voxel_size': [0.9, 0.9, 1.2, 1]})

12
{ 'sizeX' : 30, 'sizeY' : 30, 'sizeZ' : 30, 'sizeT' : 4, 'voxel_size' : [ 0.9, 0.9, 1.2, 1 ] }


Similarly, 1D or 2D volumes may be used exactly the same way.


#### The older AimsData classes

For historical reasons, another set of classes may also represent volumes. These classes are the older API in AIMS, and tend to be obsolete. 
But as they were used in many many routines and programs, they have still not been eradicated. 
Many C++ routines build volumes and actually return those older classes, so we could not really hide them, and they also have python bindings. 
These classes are `aims.AimsData_<type>`, for example [soma.aims.AimsData_FLOAT](pyaims_lowlevel.html#soma.aims.AimsData_FLOAT). 
Converting from and to [soma.aims.Volume_FLOAT](pyaims_lowlevel.html#soma.aims.Volume_FLOAT) classes is rather simple since the newer `Volume` classes are used internally in the `AimsData` API.


In [43]:
from soma import aims
# create a 4D volume of signed 16-bit ints, of size 30x30x30x4
vol = aims.Volume(30, 30, 30, 4, 'S16')
vol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
advol = aims.AimsData(vol)
# vol and advol share the same header and voxel data
vol.setValue(12, 10, 10, 20, 2)
print('advol.value(10, 10, 20, 2):', advol.value(10, 10, 20, 2))
assert(advol.value(10, 10, 20, 2) == 12)
advol.setValue(44, 12, 12, 24, 1)
print('vol.value(12, 12, 24, 1):', vol.value(12, 12, 24, 1))
assert(vol.value(12, 12, 24, 1) == 44)

advol.value(10, 10, 20, 2): 12
vol.value(12, 12, 24, 1): 44


And, in the other direction:

In [44]:
# create a 4D volume of signed 16-bit ints, of size 30x30x30x4
advol = aims.AimsData(30, 30, 30, 4, 'S16')
advol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
vol = advol.volume()
# vol and advol share the same header and voxel data
vol.setValue(12, 10, 10, 20, 2)
print('advol.value(10, 10, 20, 2):', advol.value(10, 10, 20, 2))
assert(advol.value(10, 10, 20, 2) == 12)
advol.setValue(44, 12, 12, 24, 1)
print('vol.value(12, 12, 24, 1):', vol.value(12, 12, 24, 1))
assert(vol.value(12, 12, 24, 1) == 44)

advol.value(10, 10, 20, 2): 12
vol.value(12, 12, 24, 1): 44


`AimsData` has a bit richer API, since it includes minor processing functions that have been removed from the newer `Volume` for the sake of API simplicity and minimalism.

In [45]:
# minimum / maximum
print('min:', advol.minimum(), 'at', advol.minIndex())
assert(advol.minimum() == 0)
assert(advol.minIndex() == ((0, 0, 0, 0), 0))
print('max:', advol.maximum(), 'at', advol.maxIndex())
assert(advol.maximum() == 44)
assert(advol.maxIndex() == ((12, 12, 24, 1), 44))

min: 0 at ((0, 0, 0, 0), 0)
max: 44 at ((12, 12, 24, 1), 44)


In [46]:
# clone copy
advol2 = advol.clone()
advol2.setValue(12, 4, 8, 11, 3)
# now advol and advol2 have different values
print('advol.value(4, 8, 11, 3):', advol.value(4, 8, 11, 3))
assert(advol.value(4, 8, 11, 3) == 0)
print('advol2.value(4, 8, 11, 3):', advol2.value(4, 8, 11, 3))
assert(advol2.value(4, 8, 11, 3) == 12)

advol.value(4, 8, 11, 3): 0
advol2.value(4, 8, 11, 3): 12


In [47]:
# Border handling
# Border width is th 5th parameter of AimsData constructor
advol = aims.AimsData(192, 256, 128, 1, 2, 'S16')
advol.header()['voxel_size'] = [0.9, 0.9, 1.2, 1.]
advol.fill(0)
advol.setValue(15, 100, 100, 60)
vol = advol.volume()
refvol = vol.refVolume()
# the underlying refvol is 4 voxels wider in each direction, and shifted:
print('refvol.value(100, 100, 60):', refvol.value(100, 100, 60))
assert(refvol.value(100, 100, 60) == 0)
# ... it is 0, not 15...
print('refvol.value(102, 102, 62):', refvol.value(102, 102, 62))
assert(refvol.value(102, 102, 62) == 15)
# here we get 15
# some algorithms require this border to exist, otherwise fail or crash...
from soma import aimsalgo
aimsalgo.AimsDistanceFrontPropagation(advol, 0, -1, 3, 3, 3, 10, 10)
aims.write(advol, 'distance3.nii')

refvol.value(100, 100, 60): 0
refvol.value(102, 102, 62): 15


### Meshes

#### Structure

A surfacic mesh represents a surface, as a set of small polygons (generally triangles, but sometimes quads). 
It has two main components: a vector of vertices (each vertex is a 3D point, with coordinates in millimeters), 
and a vector of polygons: each polygon is defined by the vertices it links (3 for a triangle). It also optionally has normals (unit vectors). 
In our mesh structures, there is one normal for each vertex.


In [48]:
from soma import aims
mesh = aims.read('data_for_anatomist/subject01/subject01_Lhemi.mesh')
vert = mesh.vertex()
print('vertices:', len(vert))
assert(len(vert) == 33837)
poly = mesh.polygon()
print('polygons:', len(poly))
assert(len(poly) == 67678)
norm = mesh.normal()
print('normals:', len(norm))
assert(len(norm) == 33837)

vertices: 33837
polygons: 67678
normals: 33837


To build a mesh, we can instantiate an object of type `aims.AimsTimeSurface_<n>_VOID`,
for example [soma.aims.AimsTimeSurface_3_VOID](pyaims_lowlevel.html#soma.aims.AimsTimeSurface_3_VOID), with *n* being the number of vertices by polygon. VOID means that the mesh has no texture in it (which we generally don't use, we prefer using texture as separate objects).
Then we can add vertices, normals and polygons to the mesh:

In [49]:
# build a flying saucer mesh
from soma import aims
import numpy
mesh = aims.AimsTimeSurface(3)
# a mesh has a header
mesh.header()['toto'] = 'a message in the header'
vert = mesh.vertex()
poly = mesh.polygon()
x = numpy.cos(numpy.ogrid[0.: 20] * numpy.pi / 10.) * 100
y = numpy.sin(numpy.ogrid[0.: 20] * numpy.pi / 10.) * 100
z = numpy.zeros(20)
c = numpy.vstack((x, y, z)).transpose()
vert.assign([aims.Point3df(0., 0., -40.), aims.Point3df(0., 0., 40.)] + [aims.Point3df(x) for x in c])
pol = numpy.vstack((numpy.zeros(20, dtype=numpy.int32), numpy.ogrid[3: 23], numpy.ogrid[2: 22])).transpose()
pol[19, 1] = 2
pol2 = numpy.vstack((numpy.ogrid[2: 22], numpy.ogrid[3: 23], numpy.ones(20, dtype=numpy.int32))).transpose()
pol2[19, 1] = 2
poly.assign([aims.AimsVector(x, dtype='U32',dim=3) for x in numpy.vstack((pol, pol2))])
# write result
aims.write(mesh, 'saucer.mesh')
# automatically calculate normals
mesh.updateNormals()

![Flying saucer mesh](images/saucer.png "Flying saucer mesh")


#### Modifying a mesh

In [50]:
# slightly inflate a mesh
from soma import aims
import numpy
mesh = aims.read('data_for_anatomist/subject01/subject01_Lwhite.mesh')
vert = mesh.vertex()
varr = numpy.array(vert)
norm = numpy.array(mesh.normal())
varr += norm * 2 # push vertices 2mm away along normal
vert.assign([aims.Point3df(x) for x in varr])
mesh.updateNormals()
aims.write(mesh, 'subject01_Lwhite_semiinflated.mesh')

Now look at both meshes in Anatomist...

Alternatively, without numpy, we could have written the code like this:

In [52]:
mesh = aims.read('data_for_anatomist/subject01/subject01_Lwhite.mesh')
vert = mesh.vertex()
norm = mesh.normal()
for v, n in zip(vert, norm):
    v += n * 2
mesh.updateNormals()
aims.write(mesh, 'subject01_Lwhite_semiinflated.mesh')

![Inflated mesh](images/semi_inflated.png "Inflated mesh")


#### Handling time

In AIMS, meshes are actually time-indexed dictionaries of meshes. 
This way a deforming mesh can be stored in the same object. 
To copy a timestep to another, use the following:

In [53]:
from soma import aims
mesh = aims.read('data_for_anatomist/subject01/subject01_Lwhite.mesh')
# mesh.vertex() is equivalent to mesh.vertex(0)
mesh.vertex(1).assign(mesh.vertex(0))
# same for normals and polygons
mesh.normal(1).assign(mesh.normal(0))
mesh.polygon(1).assign(mesh.polygon(0))
print('number of time steps:', mesh.size())
assert(mesh.size() == 2)

number of time steps: 2


<div class="topic">
<p><b>Exercise</b></p>
Make a deforming mesh that goes from the original mesh to 5mm away, by steps of 0.5 mm
</div>

In [54]:
from soma import aims
import numpy
mesh = aims.read('data_for_anatomist/subject01/subject01_Lwhite.mesh')
vert = mesh.vertex()
varr = numpy.array(vert)
norm = numpy.array(mesh.normal())
for i in xrange(1, 10):
    mesh.normal(i).assign(mesh.normal())
    mesh.polygon(i).assign(mesh.polygon())
    varr += norm * 0.5
    mesh.vertex(i).assign([aims.Point3df(x) for x in varr])
print('number of time steps:', mesh.size())
assert(mesh.size() == 10)
mesh.updateNormals()
aims.write(mesh, 'subject01_Lwhite_semiinflated_time.mesh')

number of time steps: 10


![Inflated mesh with timesteps](images/semi_inflated_time.png "Inflated mesh with timesteps")

#### Textures

A texture is merely a vector of values, each of them is assigned to a mesh vertex, with a one-to-one mapping, in the same order.
A texture is also a time-texture.

In [56]:
from soma import aims
tex = aims.TimeTexture('FLOAT')
t = tex[0] # time index, inserts on-the-fly
t.reserve(10) # pre-allocates memory
for i in xrange(10):
    t.append(i / 10.)
print(tex.size())
assert(len(tex) == 1)
print(tex[0].size())
assert(len(tex[0]) == 10)
print(tex[0][5])
assert(tex[0][5] == 0.5)

1
10
0.5


<div class="topic">
<p><b>Exercise</b></p>
Make a time-texture, with at each time/vertex of the previous mesh, sets the value of the underlying volume *data_for_anatomist/subject01/subject01.nii*
</div>

In [57]:
from soma import aims
mesh = aims.read('subject01_Lwhite_semiinflated_time.mesh')
vol = aims.read('data_for_anatomist/subject01/subject01.nii')
tex = aims.TimeTexture('FLOAT')
vs = vol.header()['voxel_size']
for i in range(mesh.size()):
    t = tex[i]
    vert = mesh.vertex(i)
    t.reserve(len(vert))
    for p in vert:
        t.append(vol.value(*[int(round(x / y)) for x, y in zip(p, vs)]))
aims.write(tex, 'subject01_Lwhite_semiinflated_texture.tex')

Now look at the texture on the mesh (inflated or not) in Anatomist. Compare it to a 3D fusion between the mesh and the MRI volume.

![Computed time-texture vs 3D fusion](images/texture.png "Computed time-texture vs 3D fusion")

**Bonus:** We can do the same for functional data. 
But in this case we may have a spatial transformation to apply between anatomical data and functional data 
(which may have been normalized, or acquired in a different referential).

In [58]:
from soma import aims
import numpy
mesh = aims.read('subject01_Lwhite_semiinflated_time.mesh')
vol = aims.read('data_for_anatomist/subject01/Audio-Video_T_map.nii')
# get header info from anatomical volume
f = aims.Finder()
assert(f.check('data_for_anatomist/subject01/subject01.nii'))
anathdr = f.header()
# get functional -> MNI transformation
m1 = aims.AffineTransformation3d(vol.header()['transformations'][1])
# get anat -> MNI transformation
m2 = aims.AffineTransformation3d(anathdr['transformations'][1])
# make anat -> functional transformation
anat2func = m1.inverse() * m2
# include functional voxel size to get to voxel coordinates
vs = vol.header()['voxel_size']
mvs = aims.AffineTransformation3d(numpy.diag(vs[:3] + [1.]))
anat2func = mvs.inverse() * anat2func
# now go as in the previous program
tex = aims.TimeTexture('FLOAT')
for i in xrange(mesh.size()):
    t = tex[i]
    vert = mesh.vertex(i)
    t.reserve(len(vert))
    for p in vert:
        t.append(vol.value(*[int(round(x)) for x in anat2func.transform(p)]))
aims.write(tex, 'subject01_Lwhite_semiinflated_audio_video.tex')

See how the functional data on the mesh changes across the depth of the cortex. 
This demonstrates the need to have a proper projection of functional data before dealing with surfacic functional processing.


### Buckets

"Buckets" are voxels lists. They are typically used to represent ROIs.
A BucketMap is a list of Buckets. Each Bucket contains a list of voxels coordinates.
A BucketMap is represented by the class [soma.aims.BucketMap_VOID](pyaims_lowlevel.html#soma.aims.BucketMap_VOID).

#### At the end, cleanup the temporary working directory

In [26]:
# cleanup data
import shutil
os.chdir(older_cwd)
shutil.rmtree(tuto_dir)